In [1]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [2]:
""" results_path = ['../runs_trainings/no_freeze/multilabel/weighted',
                '../runs_trainings/no_freeze/decremental_multilabel/weighted',         
                '../runs_trainings/no_freeze/incremental_decremental_multilabel/weighted',
                '../runs_trainings/no_freeze/joint_incremental_multilabel/weighted',
                '../runs_trainings/no_freeze/joint_incremental_restored_multilabel/weighted/reset',

                '../runs_trainings/freeze_backbone/joint_incremental_restored_multilabel/weighted/reset',
                '../runs_trainings/freeze_backbone/joint_incremental_multilabel/weighted/new',
                '../runs_trainings/freeze_backbone/incremental_decremental_multilabel/weighted/new',
                '../runs_trainings/freeze_backbone/decremental_multilabel/weighted/new',
                '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new',

                '../runs_trainings/fd/incremental_decremental/lambda_1',
                '../runs_trainings/fd/incremental_decremental/lambda_01',
                '../runs_trainings/fd/incremental_decremental/lambda_10',
                '../runs_trainings/fd/decremental/lambda_01',
                '../runs_trainings/fd/joint_incremental/lambda_01',
                '../runs_trainings/fd/baseline/lambda_01',


                '../runs_trainings/lwf/incremental_decremental/lambda_10/temp_1',
                '../runs_trainings/lwf/incremental_decremental/lambda_2/temp_1',
                '../runs_trainings/lwf/incremental_decremental/lambda_1/temp_1',
                '../runs_trainings/lwf/baseline/lambda_1',
                '../runs_trainings/lwf/decremental/lambda_1',
                '../runs_trainings/lwf/joint_incremental/lambda_1',
                ] """
results_path = ['../runs_trainings/from_checkpoint_sgd/no_freeze/incremental_decremental/4_4_lr',
                '../runs_trainings/from_checkpoint_sgd/no_freeze/joint_incremental',
                '../runs_trainings/from_checkpoint_sgd/no_freeze/decremental',
                '../runs_trainings/from_checkpoint_sgd/no_freeze/data_substitution',

                '../runs_trainings/from_checkpoint_sgd/freeze/incremental_decremental',
                '../runs_trainings/from_checkpoint_sgd/freeze/joint_incremental',
                '../runs_trainings/from_checkpoint_sgd/freeze/decremental',
                '../runs_trainings/from_checkpoint_sgd/freeze/data_substitution',

                '../runs_trainings/from_checkpoint_sgd/fd/incremental_decremental',
                '../runs_trainings/from_checkpoint_sgd/fd/joint_incremental',
                '../runs_trainings/from_checkpoint_sgd/fd/decremental',
                '../runs_trainings/from_checkpoint_sgd/fd/data_substitution',

                '../runs_trainings/from_checkpoint_sgd/lwf/incremental_decremental',
                '../runs_trainings/from_checkpoint_sgd/lwf/joint_incremental',
                '../runs_trainings/from_checkpoint_sgd/lwf/decremental',
                '../runs_trainings/from_checkpoint_sgd/lwf/data_substitution',

                ]
seeds = [0,1,2]


In [3]:
scalars_file_names = ['acc.out', 'forg_acc.out','forg_mean_ap.out','map_weighted.out','mean_ap.out']
multidim_file_names = ['acc_per_class.out','ap.out','forg_accuracy_per_class.out','forg_accuracy_per_subcategory.out',
                       'forg_ap_per_subcategory.out','forg_ap.out','forg_recall_per_class.out','forg_precision_per_class.out','forg_recall_per_subcategory.out',
                       'forg_precision_per_subcategory.out','precision_per_class.out','recall_per_class.out']

In [4]:
for res_path in results_path:
    statistics_save_path = os.path.join(res_path,'statistics')
    if not os.path.exists(statistics_save_path):
        os.mkdir(statistics_save_path)
    mean_std_save_path = os.path.join(statistics_save_path,'mean_std_from_seeds')
    if not os.path.exists(mean_std_save_path):
        os.mkdir(mean_std_save_path)
    mean_save_path = os.path.join(statistics_save_path,'mean_from_seeds')
    if not os.path.exists(mean_save_path):
        os.mkdir(mean_save_path)
    std_save_path = os.path.join(statistics_save_path,'std_from_seeds')
    if not os.path.exists(std_save_path):
        os.mkdir(std_save_path)

    data = {}
    for file_name in scalars_file_names:
        data[file_name] = []
        for idx_seed in seeds:
            seed_path = os.path.join(res_path, 'seed_' + str(idx_seed))
            
            for exp_dir in os.listdir(seed_path):
                exp_path = os.path.join(seed_path,exp_dir)
                logger_path = os.path.join(exp_path,'logger')
                file_path = os.path.join(logger_path,file_name)
                file_data = np.loadtxt(file_path,delimiter=',')
                data[file_name].append(file_data)
    data_mean = {}
    data_std = {}
    data_string = {}
    for file_name in scalars_file_names:
        # this is done because i have the ap in 0,1 range, want to be in percentage
        if not ((file_name == 'mean_ap.out') or (file_name == 'map_weighted.out') or (file_name == 'forg_mean_ap.out') or (file_name == 'forg_acc.out') or (file_name == 'acc.out')):
            for i in range(len(data[file_name])):
                data[file_name][i] = data[file_name][i]*100
        data_mean[file_name] = np.mean(data[file_name], axis=0)
        data_std[file_name] = np.std(data[file_name], axis=0, ddof=1)

        
        np.savetxt(os.path.join(mean_save_path,file_name), data_mean[file_name],delimiter=',',fmt='%.1f')
        np.savetxt(os.path.join(std_save_path,file_name), data_std[file_name],delimiter=',',fmt='%.1f')
        
        tmp_mean = data_mean[file_name].tolist()
        tmp_std = data_std[file_name].tolist()
        tmp_string_list = []
        for idx in range(len(tmp_mean)):
            tmp_string = "{:.1f}".format(tmp_mean[idx])+'\u00B1'+"{:.1f}".format(tmp_std[idx])
            tmp_string_list.append(tmp_string)
        data_string[file_name] = tmp_string_list
        np.savetxt(os.path.join(mean_std_save_path,file_name), data_string[file_name],delimiter=',',fmt='%s')

    data = {}
    for file_name in multidim_file_names:
        data[file_name] = []
        for idx_seed in seeds:
            seed_path = os.path.join(res_path, 'seed_' + str(idx_seed))
            
            for exp_dir in os.listdir(seed_path):
                exp_path = os.path.join(seed_path,exp_dir)
                logger_path = os.path.join(exp_path,'logger')
                file_path = os.path.join(logger_path,file_name)
                file_data = np.loadtxt(file_path,delimiter=',')
                data[file_name].append(file_data)
    data_mean = {}
    data_std = {}
    data_string = {}
    for file_name in multidim_file_names:
        # this is done because i have the ap in 0,1 range, want to be in percentage
        if not ((file_name == 'mean_ap.out') or (file_name == 'map_weighted.out') or (file_name == 'forg_mean_ap.out') or (file_name == 'forg_acc.out') or (file_name == 'acc.out')):
            for i in range(len(data[file_name])):
                data[file_name][i] = data[file_name][i]*100
        data_mean[file_name] = np.mean(data[file_name], axis=0)
        data_std[file_name] = np.std(data[file_name], axis=0, ddof=1)

        np.savetxt(os.path.join(mean_save_path,file_name), data_mean[file_name],delimiter=',',fmt='%.1f')
        np.savetxt(os.path.join(std_save_path,file_name), data_std[file_name],delimiter=',',fmt='%.1f')
        
        tmp_mean = data_mean[file_name].tolist()
        tmp_std = data_std[file_name].tolist()
        tmp_string_array = []
        for i in range(len(tmp_mean)):
            tmp_string_list = []
            for j in range(len(tmp_mean[i])):
                tmp_string = "{:.1f}".format(tmp_mean[i][j])+'\u00B1'+"{:.1f}".format(tmp_std[i][j])
                tmp_string_list.append(tmp_string)
            tmp_string_array.append(tmp_string_list)
            data_string[file_name] = tmp_string_array
            np.savetxt(os.path.join(mean_std_save_path,file_name), data_string[file_name],delimiter=',',fmt='%s')